0. Imports and Random Seed setting

In [1]:
import torch
import torchvision
import numpy as np

torch.manual_seed(42)

1. Data Loading and Preperation

In [9]:
train_data = torchvision.datasets.MNIST(root = '../data',train = True,download= True)
test_data = torchvision.datasets.MNIST(root = '../data',train = False,download=True)
#Has been ran once to obtain and store data, now download paramter has been changed to False

x_train = train_data.data.float() / 255 #Normalising Greyscale Image Pixel Intesity
y_train = train_data.targets

x_test = test_data.data.float() / 255
y_test = test_data.targets

x_train = x_train.view(-1,28*28)
x_test = x_test.view(-1,28*28)  #Flattens the images to 2d, could use -1 or 60_000 and 10_000

def one_hot(labels,num_classes = 10):
    output = torch.zeros(labels.size(0),num_classes)
    row = torch.arange(labels.size(0))
    output[row,labels] = 1.0
    return output
#One hot encode label data so it can be usefully analysed

y_test_oh = one_hot(y_test,10)
y_train_oh = one_hot(y_train,10)

def get_batches(x,y,batch_size = 64):
    idx = torch.randperm(x.size(0))
    batches = torch.tensor()
    for i in range(0,x.size(0),batch_size):
        batches_idx = idx[i,i+batch_size]
        yield x[batches_idx],y[batches_idx]
        
        

2. Building and Training MLP without High Level help

In [ ]:
w1 = torch.randn(size = [28*28,64],requires_grad= True)
w2 = torch.randn(size = [64,10],requires_grad= True)
b1 = torch.randn(size = 64,requires_grad= True)
b2 = torch.randn(size = 10,requires_grad= True)

for x,y in get_batches(x_train,y_train_oh):
    z1 = x @ w1 + b1
    relu = torch.maximum(z1,torch.zeros_like(z1))
    z2 = relu @ w2 + b2
    probs = torch.softmax(z2,dim = 1)
    shift = 1e-10 #Prevent taking log of 0 choose small shift to minimally effect results
    class_probs = torch.sum(probs * y,dim= 1)
    cross_entropy = -torch.log(class_probs + shift)
    loss = cross_entropy.mean()
    loss.backward()
    learning_rate = 0.01 
    with torch.no_grad():
        w1 = w1 - learning_rate * w1.grad
        w2 = w2 - learning_rate * w2.grad
        b1 = b1 - learning_rate * b1.grad
        b2 = b2 - learning_rate * b2.grad
    
    w1.grad.zero_()
    w2.grad.zero_()
    b1.grad.zero_()
    b2.grad.zero_()
    
    